In [ ]:
from importlib import reload
import os
import sys
import platform
import shutil
import re
import datetime
import logging
import multiprocessing
from glob import glob
from fnmatch import fnmatch
from tracker.tank import Tank
from tracker.tracker import Tracker
from tracker import utils
import cv2
import numpy as np
import pandas as pd
import flatten_dict

import tracker
reload(tracker.utils)
from tracker import utils
reload(tracker.tank)
from tracker.tank import Tank
reload(tracker.tracker)
from tracker.tracker import Tracker

In [ ]:
''' Select an output directory, a set of input files (videos to track), 
    and a settings file. '''

tracking_dir = 'output'

input_files = sorted(glob('../raw_videos2/videos/*.avi'))
# input_files = [ '../raw_videos2/videos/pa_wed_28dpf_groupA_n5_202104281846.avi' ]
input_files = [input_files[i] for i in [0,2]]

filtered_settings = utils.load_filtered_settings('settings/larval-schooling-2.xlsx')

input_list = []
for input_file in input_files:
#     print(input_file)
    d,fn       = os.path.split(input_file)
    fn,ext     = os.path.splitext(fn)
    output_dir = os.path.join(tracking_dir,fn)
    n_ind      = int(re.findall('_n(\d+)[^\d]*_',fn)[0])
    
#     # If a repaired version exists, use that instead.
#     repaired_file = os.path.join(d,'repaired',fn+'-repaired'+ext)
#     input_file = repaired_file if os.path.exists(repaired_file) else input_file
    
#     # If a trial file already exists, skip.
#     if os.path.exists(os.path.join(output_dir,'trial.pik')):
#         continue
    
    settings = utils.apply_filtered_settings(filtered_settings, fn)
    settings.update( input_video=input_file, output_dir=output_dir, 
                     n_ind=n_ind, trial_name=fn )
    input_list.append(settings)

print(f'{len(input_list)}/{len(input_files)}')

# if len(input_dict)<10:
for D in input_list:
    print(D['trial_name'])

In [ ]:
import tracker
reload(tracker.utils)
from tracker import utils
reload(tracker.tank)
from tracker.tank import Tank
reload(tracker.tracker)
from tracker.tracker import Tracker

''' Create output directories. Import or compute backgrounds. '''

if not os.path.exists(tracking_dir):
    os.mkdir(tracking_dir)

# If not None, look for pre-existing backgrounds in bkg_dir.
bkg_dir = os.path.relpath(os.path.join(tracking_dir,'../full_21-01-17')) # None # 

for D in input_list:
    utils.overprint(D['trial_name'])
#     print(D['trial_name'])
    tracker = Tracker(**D)
    
    # Create output directory.
    tracker.init_directory()
    tracker.save_settings()
    
    # Create a link to the input video in the output directory.
    # In windows, this requires special privileges.
    # Instead, write the path to the video in a text file which
    # the viewer can use to locate the video.
    input_link     = os.path.join(D['output_dir'],'raw'+ext)
    relative_input = os.path.relpath(input_file, D['output_dir'])
    if not os.path.exists(input_link):
        try:
            input_link = os.path.join(D['output_dir'], 'raw'+ext)
            os.symlink(relative_input, input_link)
        except:
#         if 'windows' in platform.system().lower():
            input_link = os.path.join(D['output_dir'], 'raw.txt')
            with open(input_link, 'r') as fh:
                fh.write(relative_input)
    
    # If background file(s) already exist, don't do anything.
    b  = D['bkg.secondary_subtraction']
    f1 = os.path.join(tracker.output_dir,'background.npz')
    f2 = os.path.join(tracker.output_dir,'background2.npz')
    if os.path.exists(f1) and ( (not b) or os.path.exists(f2) ):
        continue
    # If background file exists in bkg_dir, use it.
    try:
        d = os.path.join(bkg_dir,D['trial_name'])
        for i in ['']: #,'2']:
            for e in ['npz','png']:
                f1 = os.path.join(d,f'background{i}.{e}')
                f2 = os.path.join(D['output_dir'],f'background{i}.{e}')
                shutil.copy2(f1,f2)
    except:
        pass
    # Otherwise, compute background.
    tracker.init_video_input()
    tracker.init_background()
    tracker.init_secondary_background()
    tracker.release()

print('\nDone')

In [ ]:
''' Trace tanks. '''

# If not None, look for pre-existing tanks in tank_dir.
tank_dir = None # os.path.join(tracking_dir,'../full_21-01-17') # 

for D in input_list:
    logging.info(D['trial_name'])
    
    f = os.path.join(D['output_dir'],f'tank.pik')
    if os.path.exists(f):
        continue
    
#     try:
#         d = os.path.join(tank_dir,D['trial_name'])
#         for e in ['pik','png']:
#             f2 = os.path.join(D['output_dir'],f'tank.{e}')
#             if not os.path.exists(f2):
#                 f1 = os.path.join(d,f'tank.{e}')
#                 shutil.copy2(f1,f2)
#     except:
#         pass
    
    tracker = Tracker(**D)
    tracker.init_video_input()
    tracker.init_background()
    tracker.release()
    tracker.init_tank(thresh=4,dilate=20)

print('Done')

In [ ]:
cv2.destroyAllWindows()

In [ ]:
import tracker
reload(tracker.utils)
from tracker import utils
reload(tracker.tank)
from tracker.tank import Tank
reload(tracker.tracker)
from tracker.tracker import Tracker


def track(settings):
    globals().update(settings)
    utils.add_log_file(os.path.join(output_dir,'log.txt'))
    logging.info(utils.parindent+'Initializing')
    tracker = Tracker(**settings)
    tracker.init_all()
    tracker.save_settings()
    try:
        logging.info(utils.parindent+'Starting to track')
        tracker.set_frame(tracker.frame_start)
        n_report = max(1,int((tracker.frame_end-tracker.frame_start)/100))
        for i_frame in range(tracker.frame_start, tracker.frame_end):
            if (i_frame-tracker.frame_start)%n_report==0:
                tracker.save_trial()
                percent = (i_frame-tracker.frame_start)/(tracker.frame_end-tracker.frame_start)
                t = str(datetime.datetime.now()).split('.')
                logging.info(utils.parindent + f'Tracking: {tracker.get_current_timestamp()}, ' +
                             f'{tracker.get_percent_complete():4.1f}% complete, {t[0]}.{t[1][:2]}' )
            if not tracker.track_next_frame():
                raise Exception(f'Could not read frame {i_frame}.')
        tracker.release()
        logging.info(utils.parindent+'Saving')
        tracker.save_trial()
        logging.info(utils.parindent+'Done')
    except:
        tracker.release()
        logging.info('Failed')
        for info in sys.exc_info():
            logging.warning(info)
        %tb
    utils.reset_logging()
    return


n_threads = 1
# This multi-threading does not work in Windows. 
# If the OS is Windows, use a single thread.
if n_threads==1 or ('windows' in platform.system().lower()):
    for settings in input_list:
        utils.reset_logging()
        utils.add_log_stream(sys.stdout, overwrite=True)
        logging.info(settings['input_video']+'\n')
        track(settings)
else:
    def track_(settings):
        utils.reset_logging()
        return track(settings)
    pool = multiprocessing.Pool(n_threads)
    pool.map(track_,input_list)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

trial_files = glob(os.path.join(tracking_dir,'*/trial.pik'))
# trial_files = glob(os.path.join(tracking_dir,'Pa_Mon_7dpf_GroupA_n5_20200706_1140/trial.pik'))
display(trial_files)

trial_file = trial_files[0]
trial = utils.load_pik(trial_file)
# print(trial_file)
data,frames = trial['data'],trial['frame_list']

xy = trial['data'][:,:trial['n_ind'],:2]
xy[:,0,:].shape
for i in range(xy.shape[1]):
    plt.plot(*xy[:,i,:].T,label=f'{i}')
plt.legend(loc=(1.05,0.5))
plt.axis('equal')
plt.show()